Get rid of "ninja" win condition

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

Dict{Int64, Function} with 10 entries:
  50  => fx_melodious
  20  => fx_reciprocating
  110 => fx_otaku
  10  => fx_imitative
  90  => fx_iconoclast
  30  => fx_romantic
  120 => fx_sincere
  130 => fx_cowardly
  40  => fx_bibliophile
  100 => fx_intoxicating

In [4]:
#const MXS = 100
const MXV = 7
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

In [10]:
function sample_library(ss_prob, as, n_init = 1000000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for a1 in as
        for h in hrange
            for f in frange
                for l in lrange
                    for p in prange
                        for r in rrange
                            for s in srange
                                if rand() <= ss_prob
                                    ff = Spud("", h,f,l,p,r,s,a1)
                                    ff_i += 1
                                    ffs[ff_i] = ff
                                end
                            end
                        end                        
                    end                        
                end
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

sample_library (generic function with 2 methods)

In [11]:
struct NashEquilibrium2p
    w1::Array{Float64}
    w2::Array{Float64}
    v::Float64
end


function payoffs2(set1::Array{Spud}, set2::Array{Spud})::Array{Int64}
    n1 = length(set1)
    n2 = length(set2)
    payoffs = Array{Int64}(undef, (n1, n2))
    for i in 1:n1
        for j in 1:n2
            payoffs[i,j] = eval_battle(set1[i], set2[j])
        end
    end
    return payoffs
end

function ffp2(set1::Array{Spud}, set2::Array{Spud}, nits::Int64)::NashEquilibrium2p
    n1 = length(set1)
    n2 = length(set2)
    payoffs = payoffs2(set1, set2)
    v1 = [1 for i in 1:n1]
    v2 = [1 for i in 1:n2]
    s1 = [sum(payoffs[i, :] .* v2) for i in 1:n1]
    s2 = [sum(payoffs[:, i] .* v1) for i in 1:n2]
    for it in 1:nits
        cands1 = findall(s1 .== maximum(s1))
        i1 = rand(cands1)
        cands2 = findall(s2 .== minimum(s2))
        i2 = rand(cands2)
        v1[i1] += 1
        s2 = s2 .+ payoffs[i1, :]
        v2[i2] += 1
        s1 = s1 .+ payoffs[:, i2]
    end
    w1 = v1./sum(v1)
    w2 = v2./sum(v2)
    val = sum([payoffs[i, j] * w1[i] * w2[j] for i in 1:n1 for j in 1:n2])
    return NashEquilibrium2p(w1, w2, val)
end

function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end

ffp (generic function with 1 method)

## Battle code

In [12]:
mutable struct BattleView
    round::Int64
    own::Array{Spud}
    owncount::Array{Int64}
    deployed::Array{Bool}
    obs::Array{Spud}
    obscount::Array{Int64}
end

mutable struct AiPlayer
    genteam::Function
    pick::Function
    create::Function
end

In [13]:
n_copy = 3

function initialize_game(team1::Array{Spud}, team2::Array{Spud})::Array{BattleView}
    teams = [team1, team2]
    bvs = Array{BattleView}(undef, 0)
    for team in teams
        bv = BattleView(0, team, [n_copy for i in 1:length(team)], [false for i in 1:length(team)], [], [])
        append!(bvs, [bv])
    end
    return bvs
end

function observe_fighters(game::Array{BattleView}, choices::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].deployed[choices[i]] = true
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                0
            else
                game[j].obs = append!(game[j].obs, [ff])
                game[j].obscount = append!(game[j].obscount, [game[i].owncount[choices[i]]])
            end
        end
    end
    return game
end

function modify_fighter_counts(game::Array{BattleView}, choices::Array{Int64}, mod::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].owncount[choices[i]] = game[i].owncount[choices[i]] + mod[i]
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                ind = findall(x -> x==ff, game[j].obs)[1]
                game[j].obscount[ind] = game[i].owncount[choices[i]]
            else
                0
            end
        end
    end
    return game
end

function play_combat(game::Array{BattleView}, choices::Array{Int64}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    fighters = [game[i].own[choices[i]] for i in 1:2]
    for i in 1:2
        @assert game[i].owncount[choices[i]] >= 0
    end
    if verbose
        print("Round ")
        print(round)
        print(": #A")
        print(choices[1])
        print(" ")
        print(fighters[1])
        print(" vs #B")
        print(choices[2])
        print(" ")
        println(fighters[2])
    end
    outcome = eval_battle(fighters[1], fighters[2])
    if verbose
        if outcome == 0
            println("Tied!")
        else
            if outcome == 1
                print(fighters[1])
            end
            if outcome == -1
                print(fighters[2])
            end
            println(" wins!")
        end
    end
    game = observe_fighters(game, choices)
    if outcome > -1
        game = modify_fighter_counts(game, choices, [0, -1])
    end
    if outcome < 1
        game = modify_fighter_counts(game, choices, [-1, 0])
    end
    for i in 1:2
        game[i].round = round
    end
    return game
end

function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    game[player].own = append!(game[player].own, [ff])
    game[player].owncount = append!(game[player].owncount, [n_copy])
    game[player].deployed = append!(game[player].deployed, [false])
    if verbose
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(ff)
    end
    return game
end

function play_is_game_over(game::Array{BattleView})::Bool
    for bv in game
        if sum(bv.owncount) == 0
            return true
        end
    end
    return false
end

play_is_game_over (generic function with 1 method)

In [14]:
function generate_rand_team(env, counts, nteam)
    cc = cumsum(counts)
    team = Array{Spud}(undef, nteam)
    for i in 1:nteam
        tmp = rand(1:sum(counts))
        ind = sum(cc .< tmp)+1
        team[i] = env[ind]
    end
    return team
end

function ai0_pick(a::BattleView)::Int64
    inds = findall(a.owncount .> 0)
    return rand(inds)
end

function ai0_create(a::BattleView)::Spud
    return generate_rand_team(nash_env, counts, 1)[1]
end

function ai1_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    cands = nash_env[ebs .== maximum(ebs)]
    return rand(cands)
end

function ai2_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    if maximum(ebs) == length(ops)
        cands = nash_env[ebs .== maximum(ebs)]
        return rand(cands)
    else
        ebs2 = [eval_battle_list(ff, ops) for ff in lib]
        if maximum(ebs2) > maximum(ebs)
            cands = lib[ebs2 .== maximum(ebs2)]
            return rand_rename(rand(cands))
        else
            cands = nash_env[ebs .== maximum(ebs)]
            return rand(cands)
        end
    end
end


ai2_create (generic function with 1 method)

In [15]:
# picks Spud most likely to win
function ai_greedy_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    ind = findall(a.owncount .> 0)[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

ai_greedy_pick (generic function with 1 method)

In [16]:
# picks from spuds already deployed
function ai_conservative_pick(a::BattleView)::Int64
    own = a.own[a.owncount .> 0 .&& a.deployed]
    owninds = findall(a.owncount .> 0 .&& a.deployed)
    if length(own) == 0
        # pick greedy
        return ai_greedy_pick(a)
    end
    ops = a.obs[a.obscount .> 0]
    res = ffp2(own, ops, 100)
    ind = owninds[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

ai_conservative_pick (generic function with 1 method)

In [17]:
# picks Spud based on 1-round nash
function ai_myopic_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    cc = cumsum(res.w1)
    ind = sum(cc .< rand()) + 1
    return findall(a.owncount .> 0)[ind]
end

ai_myopic_pick (generic function with 1 method)

In [18]:
creation_rounds = [5, 10, 15]

function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    team1 = ai1.genteam()
    team2 = ai2.genteam()
    game = initialize_game(team1, team2)
    while !play_is_game_over(game)
        # play a round
        choice1 = ai1.pick(game[1])
        choice2 = ai2.pick(game[2])
        game = play_combat(game, [choice1, choice2], verbose)
        if verbose
            for i in 1:2
                println(game[i].owncount)
            end
        end
        if game[1].round in creation_rounds
            for i in 1:2
                ff = ais[i].create(game[i])
                game = play_add_spud(game, i, ff, verbose)
            end
        end
    end
    if sum(game[1].owncount) == 0
        if sum(game[2].owncount) == 0
            return 0
        else
            return -1
        end
    end
    return 1
end

play_game (generic function with 2 methods)

In [19]:
# ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ais = [ai1, ai2]
# play_game(ais, true)

In [20]:
@time lib = sample_library(1.01, [999]);

  0.070461 seconds (50 allocations: 99.105 MiB, 9.59% gc time)


In [21]:
function filter_nondominated(as::Array{Spud})::Array{Spud}
    df = spuds_to_df(as)
    mat = Array{Int64}(undef, (length(as), 6))
    mat[:, 1] = df.h
    mat[:, 2] = df.f
    mat[:, 3] = df.l
    mat[:, 4] = df.p
    mat[:, 5] = df.r
    mat[:, 6] = df.s;
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:6
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

spud_to_vec (generic function with 1 method)

## Generating p.win data by calculation

### Tensor-cdf computations

In [22]:
function computeTCDF6(tpdf::Array{Int64})::Array{Int64}
    tcdf = 0 .* tpdf
    for i1 in 1:size(tpdf)[1]
        for i2 in 1:size(tpdf)[2]
            for i3 in 1:size(tpdf)[3]
                for i4 in 1:size(tpdf)[4]
                    for i5 in 1:size(tpdf)[5]
                        for i6 in 1:size(tpdf)[6]
                            tcdf[i1,i2,i3,i4,i5,i6] = sum(tpdf[1:i1,1:i2,1:i3,1:i4,1:i5,1:i6])
                        end
                    end
                end
            end
        end    
    end
    return tcdf
end

function integrate_cdf(tcdf::Array{Int64}, mins::Array{Int64}, maxes::Array{Int64}, fixed::Array{Int64} = Array{Int64}(undef, 0))::Int64
    if sum(mins .== maxes) >0
        return 0
    end
    if length(mins) == 0
        if minimum(fixed) == 0
            return 0
        else
            return tcdf[CartesianIndex(tuple(fixed...))]
        end
    end
    new_min = mins[1]
    new_max = maxes[1]
    new_fixed1 = append!(copy(fixed), [new_min])
    new_fixed2 = append!(copy(fixed), [new_max])
    ub = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed2)
    if new_min <= 0
        return ub
    else
        lb = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed1)
        return ub - lb
    end
end

integrate_cdf (generic function with 2 methods)

In [23]:
function computeTCDF6(tpdf::Array{Float64})::Array{Float64}
    tcdf = 0 .* tpdf
    for i1 in 1:size(tpdf)[1]
        for i2 in 1:size(tpdf)[2]
            for i3 in 1:size(tpdf)[3]
                for i4 in 1:size(tpdf)[4]
                    for i5 in 1:size(tpdf)[5]
                        for i6 in 1:size(tpdf)[6]
                            tcdf[i1,i2,i3,i4,i5,i6] = sum(tpdf[1:i1,1:i2,1:i3,1:i4,1:i5,1:i6])
                        end
                    end
                end
            end
        end    
    end
    return tcdf
end

function integrate_cdf(tcdf::Array{Float64}, mins::Array{Int64}, maxes::Array{Int64}, fixed::Array{Int64} = Array{Int64}(undef, 0))::Float64
    if sum(mins .== maxes) >0
        return 0
    end
    if length(mins) == 0
        if minimum(fixed) == 0
            return 0
        else
            return tcdf[CartesianIndex(tuple(fixed...))]
        end
    end
    new_min = mins[1]
    new_max = maxes[1]
    new_fixed1 = append!(copy(fixed), [new_min])
    new_fixed2 = append!(copy(fixed), [new_max])
    ub = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed2)
    if new_min <= 0
        return ub
    else
        lb = integrate_cdf(tcdf, mins[2:end], maxes[2:end], new_fixed1)
        return ub - lb
    end
end

integrate_cdf (generic function with 4 methods)

### Computing p-win from a tensor-cdf

In [24]:
# spuds to min-max matrix
function spuds_to_mmm0(as::Array{Spud})::Array{Int64}
    df = spuds_to_df(as)
    mat = zeros(Int64, (length(as), 12))
    mat[:, 1] = df.h
    mat[:, 3] = df.f
    mat[:, 5] = df.l
    mat[:, 7] = df.p
    mat[:, 9] = df.r
    mat[:,11] = df.s;
    mat[:, 2] = df.h
    mat[:, 4] = df.f
    mat[:, 6] = df.l
    mat[:, 8] = df.p
    mat[:,10] = df.r
    mat[:,12] = df.s;
    return mat
end

function spuds_to_mmm(as::Array{Spud})::Array{Int64}
    mmm = spuds_to_mmm0(as);
    for j in [6,5,4,3,2,1]
        inds_used = zeros(Int64, size(mmm)[1]);
        mmm2 = Array{Int64}(undef, (size(mmm)[1], 12));
        n_mmm2 = 0;
        while minimum(inds_used) == 0
            ind_next = findall(inds_used .== 0)[1]
            v = mmm[ind_next, :];
            eq = ones(Int64, size(mmm)[1]);
            eq = eq .* (mmm[:, (2 * j -1)] .== mmm[:, (2 * j)])
            for k in 1:6
                if k != j
                    eq = eq .* (mmm[:, (2 * k -1)] .== v[(2 * k -1)])
                    eq = eq .* (mmm[:, (2 * k)] .== v[(2 * k)])
                end
            end
            mmsub = mmm[eq .== 1, :]
            minv = minimum(mmsub[:, 2*j])
            maxv = maximum(mmsub[:, 2*j])
            inds_used[eq .== 1] .= 1;
            if size(mmsub)[1] == (maxv - minv + 1)
                vnew = copy(v)
                vnew[2 * j - 1] = minv
                vnew[2 * j] = maxv
                n_mmm2 += 1
                mmm2[n_mmm2, :] = vnew
            else
                nsub = size(mmsub)[1]
                mmm2[(n_mmm2+1):(n_mmm2+nsub), :] = mmsub
                n_mmm2 += nsub
            end
        end
        mmm2 = mmm2[1:n_mmm2, :]
        mmm = mmm2;
    end
    return mmm
end

spuds_to_mmm (generic function with 1 method)

In [25]:
n_init = 10000
# generate a library where MNV = 1 and MXV = 3
ffs = Array{Spud}(undef, n_init)
ff_i = 0
hrange = 1:3
frange = 1:3
lrange = 1:3
prange = 1:3
rrange = 1:3
srange = 1:3
a1 = 999
for h in hrange
    for f in frange
        for l in lrange
            for p in prange
                for r in rrange
                    for s in srange
                        ff = Spud("", h,f,l,p,r,s,a1)
                        ff_i += 1
                        ffs[ff_i] = ff
                    end
                end                        
            end                        
        end
    end
end
ffs = ffs[1:ff_i]
libmax3 = ffs;

In [26]:
n_init = 10000
# generate a library where MNV = 1 and MXV = 3 (except for H, F where MXV = 4)
ffs = Array{Spud}(undef, n_init)
ff_i = 0
hrange = 1:4
frange = 1:4
lrange = 1:3
prange = 1:3
rrange = 1:3
srange = 1:3
a1 = 999
for h in hrange
    for f in frange
        for l in lrange
            for p in prange
                for r in rrange
                    for s in srange
                        ff = Spud("", h,f,l,p,r,s,a1)
                        ff_i += 1
                        ffs[ff_i] = ff
                    end
                end                        
            end                        
        end
    end
end
ffs = ffs[1:ff_i]
libmax4 = ffs;

In [27]:
n_init = 10000
# generate a library where MNV = 1 and MXV = 3 (except for H, F where MXV = 5)
ffs = Array{Spud}(undef, n_init)
ff_i = 0
hrange = 1:5
frange = 1:5
lrange = 1:3
prange = 1:3
rrange = 1:3
srange = 1:3
a1 = 999
for h in hrange
    for f in frange
        for l in lrange
            for p in prange
                for r in rrange
                    for s in srange
                        ff = Spud("", h,f,l,p,r,s,a1)
                        ff_i += 1
                        ffs[ff_i] = ff
                    end
                end                        
            end                        
        end
    end
end
ffs = ffs[1:ff_i]
libmax5 = ffs;

In [28]:
## case 1. H = F
ff = Spud("",2,2,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax3];
i_beat = libmax3[evs .== 1]
i_tie = libmax3[evs .== 0]
length(i_beat)
mmm_h_eq_f = spuds_to_mmm(i_beat);

In [29]:
## case 2. H = F+1
ff = Spud("",3,2,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax4];
i_beat = libmax4[evs .== 1]
i_tie = libmax4[evs .== 0]
length(i_beat)
mmm_h_gt_f = spuds_to_mmm(i_beat);

In [30]:
## case 3. H = F-1
ff = Spud("",2,3,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax4];
i_beat = libmax4[evs .== 1]
i_tie = libmax4[evs .== 0]
length(i_beat)
mmm_h_lt_f = spuds_to_mmm(i_beat);

In [31]:
## case 4. H > F+1
ff = Spud("",4,2,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax5];
i_beat = libmax5[evs .== 1]
i_tie = libmax5[evs .== 0]
length(i_beat)
mmm_h_mg_f = spuds_to_mmm(i_beat);

In [32]:
## case 5. H < F-1
ff = Spud("",2,4,2,2,2,2,999)
evs = [eval_battle(ff, gg) for gg in libmax5];
i_beat = libmax5[evs .== 1]
i_tie = libmax5[evs .== 0]
length(i_beat)
mmm_h_ml_f = spuds_to_mmm(i_beat);

In [33]:
function get_win_regions_vanilla_vs_vanilla(a::Spud)::Tuple{Array{Int64}, Array{Int64}}
    v = spud_to_vec(a)
    thres_s = Array{Array{Int64}}(undef, 6)

    ## case 1. H = F
    if v[1]==v[2]
        mmm = mmm_h_eq_f
        hf_thres = [-1, v[1]-1, v[1], MXV+1]    
    end

    ## case 2. H = F+1
    if v[1]==v[2]+1
        mmm = mmm_h_gt_f
        hf_thres = [-1, v[2]-1, v[2], v[1], MXV+1]
    end

    ## case 3. H = F-1
    if v[1]==v[2]-1
        mmm = mmm_h_lt_f
        hf_thres = [-1, v[1]-1, v[1], v[2], MXV+1]
    end

    ## case 4. H > F+1
    if v[1]> v[2]+1
        mmm = mmm_h_mg_f
        hf_thres = [-1, v[2]-1, v[2], v[1]-1, v[1], MXV+1]
    end

    ## case 5. H < F-1
    if v[1]< v[2]-1
        mmm = mmm_h_ml_f
        hf_thres = [-1, v[1]-1, v[1], v[2]-1, v[2], MXV+1]
    end


    thres_s[1] = hf_thres
    thres_s[2] = hf_thres
    for ii in 3:6
        thres_s[ii] = [-1, v[ii]-1, v[ii], MXV+1]
    end
    mins_raw = Array{Array{Int64}}(undef, 6)
    maxes_raw = Array{Array{Int64}}(undef, 6)
    inds_min = [1,3,5,7,9,11]
    inds_max = [2,4,6,8,10,12]
    for ii in 1:6
        mins_raw[ii] = thres_s[ii][mmm[:, inds_min[ii]]]
        maxes_raw[ii] = thres_s[ii][mmm[:, inds_max[ii]] .+ 1]
    end

    mins = reduce(hcat, mins_raw);
    maxes = reduce(hcat, maxes_raw);
    mins[mins .< 0] .= 0;
    mins[mins .> MXV] .= MXV;
    maxes[maxes .< 0] .= 0;
    maxes[maxes .> MXV] .= MXV;
    return (mins, maxes)
end

function total_evs_vanilla_vs_vanilla(tcdf::Array{Int64}, a::Spud)::Int64
    res = get_win_regions_vanilla_vs_vanilla(a)
    mins = res[1]
    maxes = res[2]
    sums = [integrate_cdf(tcdf, mins[ii, :], maxes[ii, :]) for ii in 1:size(mins)[1]];
    return (2 * sum(sums) - length(lib) + 1)
end

function total_evs_vanilla_vs_vanilla(tcdf::Array{Float64}, a::Spud)::Float64
    res = get_win_regions_vanilla_vs_vanilla(a)
    mins = res[1]
    maxes = res[2]
    sums = [integrate_cdf(tcdf, mins[ii, :], maxes[ii, :]) for ii in 1:size(mins)[1]];
    return (2 * sum(sums) - length(lib) + 1)
end


total_evs_vanilla_vs_vanilla (generic function with 2 methods)

### Unit-testing

In [34]:
# # TODO
# for iiiii in 1:10
#     a = rand(lib)
#     v = spud_to_vec(a)
#     evs = [eval_battle(a, gg) for gg in lib];
#     i_beat = lib[evs .== 1];
#     i_tie = lib[evs .== 0];
#     print(length(i_tie))
#     i_lose = lib[evs .== -1];
#     #mean(evs)
#     res = get_win_regions_vanilla_vs_vanilla(a)
#     mins = res[1]
#     maxes = res[2]
#     for iiii in 1:1000
#         v2 = spud_to_vec(rand(i_lose))
#         min_cond = reduce((x,y) -> x .&& y, [v2[ii] .> mins[:,ii] for ii in 1:6])
#         max_cond = reduce((x,y) -> x .&& y, [v2[ii] .<= maxes[:,ii] for ii in 1:6])
#         if sum(min_cond .&& max_cond) != 0
#             println(v2)
#         end
#     end
#     for iiii in 1:1000
#         v2 = spud_to_vec(rand(i_beat))
#         min_cond = reduce((x,y) -> x .&& y, [v2[ii] .> mins[:,ii] for ii in 1:6])
#         max_cond = reduce((x,y) -> x .&& y, [v2[ii] .<= maxes[:,ii] for ii in 1:6])
#         if sum(min_cond .&& max_cond) != 1
#             println(v2)
#         end
#     end
# end

In [35]:
# for iiiii in 1:1000
#     a = rand(lib)
#     evs = [eval_battle(a, gg) for gg in lib];
#     if sum(evs) != total_evs_vanilla_vs_vanilla(tcdf, a)
#         println(a)
#     end
# end

### generate p.win data and iterate

In [36]:
tpdf = ones(Int64, (MXV,MXV,MXV,MXV,MXV,MXV));
@time tcdf = computeTCDF6(tpdf);

  1.805558 seconds (941.49 k allocations: 3.657 GiB, 25.63% gc time, 13.99% compilation time)


In [37]:
@time scores0 = [total_evs_vanilla_vs_vanilla(tcdf, a) for a in lib]./length(lib);

 34.487524 seconds (697.96 M allocations: 46.678 GiB, 11.53% gc time, 2.64% compilation time)


In [38]:
scores0r = permutedims(reshape(scores0, (MXV,MXV,MXV,MXV,MXV,MXV)), (6,5,4,3,2,1));

In [39]:
inds = findall(scores0r .> -0.01 .&& scores0r .< 0.01)
nash_env = map(x -> Spud("",x[1],x[2],x[3],x[4],x[5],x[6],ability_none), inds);

In [40]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [41]:
@time counts = ffp(nash_env, 100000);

  3.097185 seconds (43.31 M allocations: 5.344 GiB, 21.41% gc time, 5.16% compilation time)


In [42]:
ne = nash_env[counts .> 10];
ct = counts[counts .> 10];
df = spuds_to_df(ne)
df[:, :count] = ct
df[sortperm(-df.count)[1:10], :]

Row,name,h,f,l,p,r,s,a1,count
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,,2,3,2,6,6,6,999,1273
2,,4,2,3,6,5,6,999,1268
3,,5,5,6,1,1,5,999,1164
4,,4,1,7,7,2,1,999,1061
5,,6,1,4,6,5,3,999,1024
6,,1,5,5,3,7,3,999,1010
7,,2,6,5,3,7,2,999,965
8,,1,6,5,7,4,1,999,939
9,,6,5,2,3,4,6,999,932


In [43]:
get_payoffs(ne[sortperm(-df.count)[1:10]])

10×10 Matrix{Int64}:
  0  -1  -1  -1  -1   1   1   1   1   1
  1   0  -1  -1  -1   1   1   1   1   1
  1   1   0  -1   1   1  -1  -1  -1  -1
  1   1   1   0   1  -1  -1   1  -1  -1
  1   1  -1  -1   0  -1  -1   1   1   1
 -1  -1  -1   1   1   0  -1   1   1   1
 -1  -1   1   1   1   1   0   1   1   1
 -1  -1   1  -1  -1  -1  -1   0   1  -1
 -1  -1   1   1  -1  -1  -1  -1   0  -1
 -1  -1   1   1  -1  -1  -1   1   1   0

In [44]:
wts = map(x -> exp(-abs(x)/10), scores0);
wts = wts./sum(wts) .* length(lib);
println(maximum(scores0[wts.>1]))
println(minimum(scores0[wts.>1]))


0.4035393416008636
-0.4035393416008636


#### iterations

In [45]:
using MAT

In [46]:
allscores = [scores0]
scores = scores0;
for i in 1:10
    wts = map(x -> exp(-400*abs(x)^2), scores)
    wts = wts./sum(wts) .* length(lib);
    tpdf = permutedims(reshape(wts, (MXV,MXV,MXV,MXV,MXV,MXV)), (6,5,4,3,2,1));
    tcdf = computeTCDF6(tpdf);
    @time scores = [total_evs_vanilla_vs_vanilla(tcdf, a) for a in lib]./length(lib);
    append!(allscores, [scores]);
    println(sum(wts .> 1))
    println(maximum(scores[wts.>1]))
    println(minimum(scores[wts.>1]))
end
iscores = map(x -> convert(Int64, floor(x)), scores .* 100 .+ 100);
iscores = permutedims(reshape(iscores, (MXV,MXV,MXV,MXV,MXV,MXV)), (6,5,4,3,2,1));
matwrite("temp_desE_mxv7_scores400.mat", Dict("scores" => iscores); compress = true)

 34.793889 seconds (699.40 M allocations: 46.622 GiB, 11.80% gc time, 0.59% compilation time)
13302
0.2587855700365128
-0.244620658906381
 34.450331 seconds (699.27 M allocations: 46.614 GiB, 11.68% gc time)
9078
0.16927450705936845
-0.17390129638129503
 34.411298 seconds (699.27 M allocations: 46.614 GiB, 11.57% gc time)
9412
0.12861541991965064
-0.1251918406739253
 34.525625 seconds (699.27 M allocations: 46.614 GiB, 11.56% gc time)
9501
0.11026980898306676
-0.1152294578634827
 34.524869 seconds (699.27 M allocations: 46.614 GiB, 11.59% gc time)
9450
0.10412776289845108
-0.10422221060470593
 34.632230 seconds (699.27 M allocations: 46.614 GiB, 11.60% gc time)
9460
0.09831771997399603
-0.09642741576659124
 34.573725 seconds (699.27 M allocations: 46.614 GiB, 11.63% gc time)
9461
0.09418199399969479
-0.09413136729031554
 32.728699 seconds (699.27 M allocations: 46.614 GiB, 12.27% gc time)
9472
0.09234918002380688
-0.09338020553502248
 34.412466 seconds (699.27 M allocations: 46.614 GiB

## Evaluating cost tolerance

In [47]:
using MAT
scores = matread("temp_desE_mxv7_scores400.mat")["scores"];

In [48]:
function get_spuds_with_value(scores::Array{Int64}, v::Int64)::Array{Spud}
    inds = findall(scores .== v)
    return map(x -> Spud("spud",x[1],x[2],x[3],x[4],x[5],x[6],ability_none), inds)
end


get_spuds_with_value (generic function with 1 method)

In [49]:
setA = get_spuds_with_value(scores, 100)
setB = get_spuds_with_value(scores, 90)
@time rf = ffp2(setA, setB, 100000);
rf.v

  1.741580 seconds (6.45 M allocations: 2.990 GiB, 34.49% gc time, 23.23% compilation time)


0.10320558966909199

In [50]:
df = spuds_to_df(setA)
df[:, :w] = rf.w1
println(df[sortperm(-rf.w1), :][1:20, :])

20×9 DataFrame
 Row │ name    h      f      l      p      r      s      a1     w          
     │ String  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │ spud        2      7      7      7      3      1    999  0.0118279
   2 │ spud        1      1      6      2      5      6    999  0.0111415
   3 │ spud        1      3      3      6      7      3    999  0.0108928
   4 │ spud        1      4      6      5      2      5    999  0.0105048
   5 │ spud        6      6      4      4      4      1    999  0.0102661
   6 │ spud        6      6      4      2      1      7    999  0.0101965
   7 │ spud        2      6      5      3      7      2    999  0.0101666
   8 │ spud        4      5      1      6      7      1    999  0.0101467
   9 │ spud        1      2      6      2      4      7    999  0.00941059
  10 │ spud        5      2      4      5      6      2    999  0.00935091
  11 │ spud    

In [ ]:
df = spuds_to_df(setB)
df[:, :w] = rf.w2
println(df[sortperm(-rf.w2), :][1:20, :])

In [ ]:
setA = get_spuds_with_value(scores, 100)
setB = get_spuds_with_value(scores, 95)
@time rf = ffp2(setA, setB, 100000);
rf.v

In [ ]:
setA = get_spuds_with_value(scores, 100)
setB = get_spuds_with_value(scores, 80)
@time rf = ffp2(setA, setB, 100000);
rf.v

In [ ]:
setA = get_spuds_with_value(scores, 90)
setB = get_spuds_with_value(scores, 80)
@time rf = ffp2(setA, setB, 100000);
rf.v

In [ ]:
setA = get_spuds_with_value(scores, 90)
setB = get_spuds_with_value(scores, 85)
@time rf = ffp2(setA, setB, 100000);
rf.v

In [ ]:
setA = get_spuds_with_value(scores, 92)
setB = get_spuds_with_value(scores, 88)
@time rf = ffp2(setA, setB, 100000);
rf.v

In [ ]:
setA = get_spuds_with_value(scores, 82)
setB = get_spuds_with_value(scores, 78)
@time rf = ffp2(setA, setB, 100000);
rf.v

In [ ]:
setA = get_spuds_with_value(scores, 72)
setB = get_spuds_with_value(scores, 68)
@time rf = ffp2(setA, setB, 100000);
rf.v

In [ ]:
setA = get_spuds_with_value(scores, 62)
setB = get_spuds_with_value(scores, 58)
@time rf = ffp2(setA, setB, 100000);
rf.v

In [ ]:
df = spuds_to_df(setA)
df[:, :w] = rf.w1
println(df[sortperm(-rf.w1), :][1:20, :])

In [ ]:
df = spuds_to_df(setB)
df[:, :w] = rf.w2
println(df[sortperm(-rf.w2), :][1:20, :])

## Combo search

In [60]:
lib = get_spuds_with_value(scores, 25)

611-element Vector{Spud}:
 Spud("spud", 5, 7, 3, 3, 1, 1, 999)
 Spud("spud", 2, 6, 7, 3, 1, 1, 999)
 Spud("spud", 4, 5, 4, 4, 1, 1, 999)
 Spud("spud", 3, 4, 5, 4, 1, 1, 999)
 Spud("spud", 2, 4, 7, 4, 1, 1, 999)
 Spud("spud", 7, 5, 2, 5, 1, 1, 999)
 Spud("spud", 4, 4, 4, 5, 1, 1, 999)
 Spud("spud", 2, 7, 5, 5, 1, 1, 999)
 Spud("spud", 2, 5, 6, 5, 1, 1, 999)
 Spud("spud", 2, 3, 7, 5, 1, 1, 999)
 Spud("spud", 5, 7, 1, 6, 1, 1, 999)
 Spud("spud", 6, 5, 2, 6, 1, 1, 999)
 Spud("spud", 5, 4, 3, 6, 1, 1, 999)
 ⋮
 Spud("spud", 1, 3, 3, 1, 3, 7, 999)
 Spud("spud", 1, 3, 1, 2, 3, 7, 999)
 Spud("spud", 3, 2, 1, 3, 3, 7, 999)
 Spud("spud", 3, 1, 1, 5, 3, 7, 999)
 Spud("spud", 4, 1, 1, 5, 3, 7, 999)
 Spud("spud", 5, 1, 1, 5, 3, 7, 999)
 Spud("spud", 4, 1, 2, 1, 4, 7, 999)
 Spud("spud", 1, 1, 3, 1, 4, 7, 999)
 Spud("spud", 3, 1, 1, 4, 4, 7, 999)
 Spud("spud", 4, 1, 1, 4, 4, 7, 999)
 Spud("spud", 5, 1, 1, 4, 4, 7, 999)
 Spud("spud", 6, 1, 1, 4, 4, 7, 999)

In [61]:
function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

mean([sum(spud_to_vec(ff)) for ff in lib]), minimum([sum(spud_to_vec(ff)) for ff in lib])

(19.245499181669395, 15)

In [66]:
libi[[sum(spud_to_vec(ff)) for ff in lib].==minimum([sum(spud_to_vec(ff)) for ff in lib])]

4-element Vector{Int64}:
 315321
 315141
 315312
 315114

In [62]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [63]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

payoffs = get_payoffs(lib);
libi = map(spud2int, lib)
sort(libi)

611-element Vector{Int64}:
 111565
 111655
 111745
 111773
 112374
 112455
 112545
 112563
 112653
 112734
 113147
 113174
 114263
      ⋮
 742323
 742341
 742521
 751531
 752115
 752511
 761125
 761521
 762114
 762141
 771133
 772113

In [64]:
nlib = length(lib);
combos = Array{Int64}(undef, (10000,3))
ord = sortperm([rand() for i in 1:nlib])
#ord = [i for i in 1:nlib]
nc = 0
# 3-combos
for i in ord[1:(nlib-2)]
    print(i)
    print(" ")
    pi = payoffs[i,:]
    for j in ord[(i+1):(nlib-1)]
        pj = payoffs[j, :]
        pij = [max(pi[a], pj[a]) for a in 1:nlib]
        if sum(pij .== 1) > 0
            #print(".")
            for k in ord[(j+1):nlib]
                pk = payoffs[k, :]
                pijk = [max(pi[a], pj[a], pk[a]) for a in 1:nlib]
                if minimum(pijk)==1
                    nc += 1
                    combos[nc, :] = libi[[i,j,k]]
                    println("")
                    println("----3-COMBO!-----")
                    println(libi[[i,j,k]])
                end
            end
        end
    end
end
combos = combos[1:nc,:]

220 133 92 217 247 387 106 253 62 467 10 288 421 89 313 
----3-COMBO!-----
[214533, 332245, 324315]

----3-COMBO!-----
[214533, 324315, 332245]
596 94 581 610 593 13 349 570 379 416 
----3-COMBO!-----
[123534, 214353, 334225]
474 448 560 176 393 505 414 
----3-COMBO!-----
[124434, 731254, 444232]

----3-COMBO!-----
[124434, 132543, 334225]

----3-COMBO!-----
[124434, 531254, 444232]

----3-COMBO!-----
[124434, 444232, 731254]

----3-COMBO!-----
[124434, 444232, 531254]
301 135 3 
----3-COMBO!-----
[454411, 231534, 214353]

----3-COMBO!-----
[454411, 214353, 231534]
289 469 534 120 346 
----3-COMBO!-----
[132363, 314424, 244531]

----3-COMBO!-----
[132363, 244531, 314424]
132 374 49 51 282 226 266 
----3-COMBO!-----
[434313, 231534, 214353]

----3-COMBO!-----
[434313, 214353, 231534]
16 548 506 336 
----3-COMBO!-----
[132453, 651531, 324334]

----3-COMBO!-----
[132453, 314424, 244531]

----3-COMBO!-----
[132453, 244531, 314424]

----3-COMBO!-----
[132453, 324334, 751531]

----3-COMBO!--


----3-COMBO!-----
[343253, 314424, 244531]

----3-COMBO!-----
[343253, 314424, 234532]

----3-COMBO!-----
[343253, 244531, 314424]

----3-COMBO!-----
[343253, 234532, 314424]
396 
----3-COMBO!-----
[314424, 443351, 231155]

----3-COMBO!-----
[314424, 132543, 234235]

----3-COMBO!-----
[314424, 233263, 244531]

----3-COMBO!-----
[314424, 343253, 244531]

----3-COMBO!-----
[314424, 343253, 234532]

----3-COMBO!-----
[314424, 132354, 244531]

----3-COMBO!-----
[314424, 244531, 233254]

----3-COMBO!-----
[314424, 244531, 432272]

----3-COMBO!-----
[314424, 244531, 452252]

----3-COMBO!-----
[314424, 244531, 233263]

----3-COMBO!-----
[314424, 244531, 343253]

----3-COMBO!-----
[314424, 244531, 132354]

----3-COMBO!-----
[314424, 244531, 442262]

----3-COMBO!-----
[314424, 442262, 244531]

----3-COMBO!-----
[314424, 731245, 423353]

----3-COMBO!-----
[314424, 234532, 343253]

----3-COMBO!-----
[314424, 423353, 731245]

----3-COMBO!-----
[314424, 423353, 531245]

----3-COMBO!-----
[314424, 

201×3 Matrix{Int64}:
 214533  332245  324315
 214533  324315  332245
 123534  214353  334225
 124434  731254  444232
 124434  132543  334225
 124434  531254  444232
 124434  444232  731254
 124434  444232  531254
 454411  231534  214353
 454411  214353  231534
 132363  314424  244531
 132363  244531  314424
 434313  231534  214353
      ⋮          
 731245  423353  314424
 464212  231534  214353
 464212  214353  231534
 234532  343253  314424
 234532  314424  343253
 423353  314424  731245
 423353  314424  531245
 423353  314424  631245
 531245  423353  314424
 631245  423353  314424
 424711  231534  214353
 424711  214353  231534